In [1]:
import numpy as np

def calculate_distance(city1, city2):
    """Calculates the Euclidean distance between two cities."""
    return np.linalg.norm(np.array(city1) - np.array(city2))


In [2]:

def calculate_total_distance(route, cities):
    """Calculates the total distance of a given route."""
    total_dist = 0
    for i in range(len(route)):
        total_dist += calculate_distance(cities[route[i]], cities[route[(i + 1) % len(route)]])
    return total_dist


In [3]:

def two_opt_swap(route, i, k):
    """Performs a 2-opt swap on the route."""
    new_route = route[:i] + route[k:i-1:-1] + route[k+1:]
    return new_route


In [ ]:

def two_opt(cities, initial_route=None):
    """
    Implements the 2-opt algorithm to find a shorter TSP tour.

    Args:
        cities (list of tuples): A list of city coordinates (e.g., [(x1, y1), (x2, y2), ...]).
        initial_route (list, optional): An initial route (permutation of city indices).
                                         If None, a random initial route is generated.

    Returns:
        list: The optimized route (permutation of city indices).
    """
    num_cities = len(cities)

    if initial_route is None:
        current_route = list(range(num_cities))
        np.random.shuffle(current_route)
    else:
        current_route = list(initial_route)
    improved = True
    while improved:
        improved = False
        current_best_distance = calculate_total_distance(current_route, cities)

        for i in range(1, num_cities - 1):
            for k in range(i + 1, num_cities):
                new_route = two_opt_swap(current_route, i, k)
                new_distance = calculate_total_distance(new_route, cities)

                if new_distance < current_best_distance:
                    current_route = new_route
                    current_best_distance = new_distance
                    improved = True
                    # Restart the search from the beginning with the improved route
                    break 
            if improved:
                break
    return current_route


In [57]:

# Example usage:
cities_coords = [(0, 0), (1, 5), (4, 2), (7, 8), (9, 1), (2, 7)]

# Run 2-opt
optimized_route = two_opt(cities_coords)
optimized_distance = calculate_total_distance(optimized_route, cities_coords)

print("Optimized Route:", optimized_route)
print("Optimized Distance:", optimized_distance)

# You can also provide an initial route
initial_route_example = [5, 4, 2, 3, 1, 0]
optimized_route_with_initial = two_opt(cities_coords, initial_route=initial_route_example)
optimized_distance_with_initial = calculate_total_distance(optimized_route_with_initial, cities_coords)
print("Optimized Route (with initial):", optimized_route_with_initial)
print("Optimized Distance (with initial):", optimized_distance_with_initial)

Optimized Route: [2, 4, 3, 5, 1, 0]
Optimized Distance: 29.285372362558242
Optimized Route (with initial): [5, 1, 0, 2, 4, 3]
Optimized Distance (with initial): 29.285372362558242


In [61]:
"""
Branch-and-Cut for TSP (educational implementation)

- Uses PuLP (CBC) to solve LP relaxations.
- Cutting planes: subtour elimination constraints (lazy-style).
- Branching: pick most fractional x[i,j] and branch x=0 / x=1.
- Node selection: priority queue by node.lower_bound (best-first).

Works well for small instances (n <= ~10-12). For larger n, use Concorde / Gurobi / CPLEX.
"""
import copy
import math
import heapq
from collections import deque

import pulp


EPS = 1e-9
INT_TOL = 1e-5  # tolerance to treat continuous value as integer


class TSPProblem:
    def __init__(self, dist):
        """
        dist: NxN matrix (list of lists) with dist[i][j] (diagonal can be 0 or large)
        We use a directed formulation: variables x[i,j] represent arc i->j.
        """
        self.dist = dist
        self.n = len(dist)


class Node:
    def __init__(self, fixings=None, cuts=None):
        """
        fixings: list of (i, j, val) meaning constraint x[i,j] == val (val 0 or 1)
        cuts: list of subtour cuts; each cut is set S (list or set of nodes)
              meaning sum_{i in S, j in S} x[i,j] <= |S| - 1
        """
        self.fixings = [] if fixings is None else list(fixings)
        self.cuts = [] if cuts is None else list(cuts)
        self.bound = None  # LP objective value (lower bound)
        self.solution = None  # dict (i,j) -> value from LP
        self.is_integral = False

    def copy(self):
        return Node(fixings=copy.deepcopy(self.fixings), cuts=copy.deepcopy(self.cuts))


def build_lp(problem: TSPProblem, node: Node, relax=True):
    """
    Build a PuLP LP problem for the TSP with the node's fixings and cuts.
    If `relax`==True variables are continuous [0,1]. If False (not used here) they'd be Integer.
    Returns (pulp_problem, var_dict) where var_dict[(i,j)] -> pulp variable.
    """
    n = problem.n
    dist = problem.dist

    prob = pulp.LpProblem("TSP_relax", pulp.LpMinimize)
    # create variables
    vars = {}
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            name = f"x_{i}_{j}"
            if relax:
                vars[(i, j)] = pulp.LpVariable(name, lowBound=0, upBound=1, cat="Continuous")
            else:
                vars[(i, j)] = pulp.LpVariable(name, lowBound=0, upBound=1, cat="Binary")

    # Objective
    prob += pulp.lpSum(dist[i][j] * vars[(i, j)] for (i, j) in vars)

    # degree constraints (one outgoing, one incoming)
    for i in range(n):
        prob += pulp.lpSum(vars[(i, j)] for j in range(n) if j != i) == 1  # leave i once
    for j in range(n):
        prob += pulp.lpSum(vars[(i, j)] for i in range(n) if i != j) == 1  # enter j once

    # apply fixings (branch constraints)
    for (i, j, val) in node.fixings:
        # fix variable exactly to val
        if val == 0:
            prob += vars[(i, j)] <= 0
        else:
            # val == 1
            prob += vars[(i, j)] >= 1
            prob += vars[(i, j)] <= 1

    # apply subtour elimination cuts (for sets S)
    for S in node.cuts:
        # sum x[i,j] for i in S, j in S <= |S| - 1
        prob += pulp.lpSum(vars[(i, j)] for i in S for j in S if i != j) <= len(S) - 1

    return prob, vars


def solve_lp(problem: TSPProblem, node: Node, time_limit=None, msg=False):
    """
    Solve LP relaxation for the node (builds model, runs solver).
    Fills node.bound and node.solution and node.is_integral.
    Returns True if solved to optimality, False if infeasible.
    """
    prob_lp, vars = build_lp(problem, node, relax=True)

    # solver configuration
    solver = pulp.PULP_CBC_CMD(msg=msg, timeLimit=time_limit, gapRel=1e-9)
    res = prob_lp.solve(solver)
    status = pulp.LpStatus[res]
    if status == "Infeasible" or status == "Undefined":
        return False

    # retrieve solution
    sol = {}
    for (i, j), v in vars.items():
        val = pulp.value(v)
        if val is None:
            val = 0.0
        sol[(i, j)] = float(val)

    obj = pulp.value(prob_lp.objective)
    node.bound = float(obj)
    node.solution = sol

    # test integrality on integer variables (all x's)
    is_int = True
    for val in sol.values():
        if abs(val - round(val)) > INT_TOL:
            is_int = False
            break
    node.is_integral = is_int
    return True


def extract_subtours_from_integer_solution(node: Node, n):
    """
    Given an integer solution node.solution with x[(i,j)] in {0,1},
    extract subtours as list of lists.
    Works by following arcs.
    """
    succ = [-1] * n
    for (i, j), val in node.solution.items():
        if val > 0.5:
            succ[i] = j

    visited = [False] * n
    subtours = []
    for i in range(n):
        if visited[i]:
            continue
        cur = i
        cycle = []
        while not visited[cur]:
            visited[cur] = True
            cycle.append(cur)
            cur = succ[cur]
            if cur == -1:
                # invalid/incomplete tour (shouldn't happen for valid integer solution)
                break
        subtours.append(cycle)
    return subtours


def find_most_fractional_var(node: Node):
    """
    Choose a branching variable: pick the variable with largest fractional distance from nearest integer.
    Returns (i,j,val) where val is fractional value in (0,1).
    """
    best = None
    max_frac = -1.0
    for (i, j), v in node.solution.items():
        frac_part = abs(v - round(v))
        if frac_part > max_frac and frac_part > INT_TOL:
            max_frac = frac_part
            best = (i, j, v)
    return best  # may be None if all near-integer


def tour_cost_from_solution(node: Node, problem: TSPProblem):
    """
    Compute cost of integer tour (assumes x's are 0/1) by summing dist over chosen arcs.
    """
    total = 0.0
    for (i, j), v in node.solution.items():
        if v > 0.5:
            total += problem.dist[i][j]
    return total


def branch_and_cut_tsp(problem: TSPProblem, time_limit_per_lp=None, verbose=False):
    """
    Main Branch-and-Cut driver for TSP.
    Returns (best_cost, best_tour_list)
    """
    # initial root node: no fixings, no cuts
    root = Node()

    # priority queue ordered by bound (lower bound), min-heap of (bound, node_id, node)
    # We'll fill bound after solving root. Use a counter to avoid tie issues.
    pq = []
    counter = 0

    best_cost = math.inf
    best_tour = None

    # Solve root LP
    ok = solve_lp(problem, root, time_limit=time_limit_per_lp, msg=False)
    if not ok:
        raise RuntimeError("Root LP infeasible")

    counter += 1
    heapq.heappush(pq, (root.bound, counter, root))

    # node processing loop
    while pq:
        bound, _, node = heapq.heappop(pq)
        # pruning by incumbent
        if bound >= best_cost - 1e-12:
            if verbose:
                print(f"Pruning node with bound {bound:.6f} >= best {best_cost:.6f}")
            continue

        # ensure node has up-to-date LP solution (may have been built earlier)
        if node.solution is None:
            ok = solve_lp(problem, node, time_limit=time_limit_per_lp, msg=False)
            if not ok:
                continue

        # If integral, check for subtours
        if node.is_integral:
            subtours = extract_subtours_from_integer_solution(node, problem.n)
            if len(subtours) == 1 and len(subtours[0]) == problem.n:
                # full tour found
                cost = tour_cost_from_solution(node, problem)
                if cost < best_cost - 1e-12:
                    best_cost = cost
                    # reconstruct tour
                    tour = []
                    # start from 0 and follow succ
                    succ = [-1] * problem.n
                    for (i, j), v in node.solution.items():
                        if v > 0.5:
                            succ[i] = j
                    cur = 0
                    tour.append(cur)
                    for _ in range(problem.n - 1):
                        cur = succ[cur]
                        tour.append(cur)
                    tour.append(0)
                    best_tour = tour
                    if verbose:
                        print(f"New incumbent cost {best_cost:.6f}, tour {best_tour}")
                # no need to branch this node
                continue
            else:
                # integer solution but multiple subtours -> add cuts to the node and re-solve
                if verbose:
                    print("Integer solution has subtours:", subtours)
                for S in subtours:
                    if len(S) == problem.n:
                        continue
                    # add cut S to node
                    node.cuts.append(list(S))
                # re-solve node with added cuts
                ok = solve_lp(problem, node, time_limit=time_limit_per_lp, msg=False)
                if not ok:
                    continue
                # push back to pq
                counter += 1
                heapq.heappush(pq, (node.bound, counter, node))
                continue

        # Non-integral node: try to add cuts? (we implement lazy cuts only when integer)
        # If fractional, we branch (if bound promising)
        if node.bound >= best_cost - 1e-12:
            continue

        # choose branching variable
        choice = find_most_fractional_var(node)
        if choice is None:
            # numerical weirdness: treat as integer?
            if verbose:
                print("No fractional var found but node not integral; skipping")
            continue
        (bi, bj, bval) = choice
        if verbose:
            print(f"Branching on var x[{bi},{bj}] = {bval:.6f}, node bound {node.bound:.6f}")

        # child 1: x[bi,bj] == 0
        child0 = node.copy()
        child0.fixings.append((bi, bj, 0))
        ok0 = solve_lp(problem, child0, time_limit=time_limit_per_lp, msg=False)
        if ok0 and child0.bound < best_cost - 1e-12:
            counter += 1
            heapq.heappush(pq, (child0.bound, counter, child0))

        # child 2: x[bi,bj] == 1
        child1 = node.copy()
        child1.fixings.append((bi, bj, 1))
        ok1 = solve_lp(problem, child1, time_limit=time_limit_per_lp, msg=False)
        if ok1 and child1.bound < best_cost - 1e-12:
            counter += 1
            heapq.heappush(pq, (child1.bound, counter, child1))

    return best_cost, best_tour


# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    # small example (symmetric distances), we use directed formulation but symmetric matrix
    dist = [
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0],
    ]

    prob = TSPProblem(dist)
    best_cost, best_tour = branch_and_cut_tsp(prob, time_limit_per_lp=10, verbose=True)
    print("Result:")
    print("Best cost:", best_cost)
    print("Tour:", best_tour)


Integer solution has subtours: [[0, 2], [1, 3]]
New incumbent cost 80.000000, tour [0, 1, 3, 2, 0]
Result:
Best cost: 80.0
Tour: [0, 1, 3, 2, 0]
